In [1]:
# 依赖：
# !pip install sentencepiece
# !pip install jieba
# !pip install regex
# !pip install tensorflow
# !pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf

from tokenization_jieba import JIEBATokenizer

In [4]:
print(tf.__version__)

2.6.0


In [5]:
print(hub.__version__)

0.12.0


In [6]:
tokenizer = JIEBATokenizer(
    'PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.vocab',
    'PanGu-Alpha-GPU/panguAlpha_pytorch/megatron/tokenizer/bpe_4w_pcl/vocab.model')

In [7]:
gpt = hub.load('./pangu-2.6B-tf2/')

In [11]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_k=50, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_k: 只选择前k个，如果为0则为无限制
    top_p: token的概率排在这以上才有效
    temperature: 温度，如果为1.0则相当于无效
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_k=tf.constant(top_k, tf.int32),
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode([int(x) for x in s]).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子

In [12]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_k=50, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [14]:
ask_gpt('唐伯虎是哪个朝代的人？')

['明朝', '明朝', '明朝']

In [15]:
ask_gpt('世界上最帅的是谁？')

['爱因斯坦', '迈克尔·杰克逊', '迈克尔·杰克逊']

In [16]:
ask_gpt('世界上最聪明的人是谁？')

['爱因斯坦', '爱因斯坦', '爱因斯坦']

In [17]:
ask_gpt('李清照是哪个朝代的人？')

['宋朝', '宋朝', '宋朝']

In [18]:
ask_gpt('关公和秦琼谁厉害？')

['秦琼厉害', '秦琼', '秦琼']

In [19]:
ask_gpt('如何赚一百万美元？')

['从华盛顿的一家饭店偷钱', '找一个女明星', '去赌场赌博']

In [20]:
ask_gpt('烧烤和火锅哪个好吃？')

['火锅好吃,烧烤不好吃', '火锅<eot>中国的首都是哪里?', '火锅']

In [21]:
ask_gpt('如何成为老师？')

['教师', '用一个字回答问题', '向学生问问题']

In [22]:
ask_gpt('佩奇是什么动物？')

['羊', '老鼠', '猴子']

In [23]:
ask_gpt('大象有几条腿？')

['一条', '四条腿', '四条腿']

In [24]:
ask_gpt('姚明有多高？')

['2.15米', '2.16米', '2.08米']

In [25]:
ask_gpt('汤姆和杰瑞谁厉害？')

['汤姆厉害', '汤姆厉害,杰瑞更厉害', '汤姆厉害,杰瑞不厉害']

In [26]:
ask_gpt('野原新之助能打过樱桃子吗？')

['不知道', '不可能', '不能']

In [27]:
ask_gpt('学音乐和学画画哪个好？')

['都好', '艺术', '画画好']

In [28]:
ask_gpt('中文和英文哪个难学？')

['中文难学', '中文更难学', '中文难学']

In [29]:
ask_gpt('python和java哪个难？')

['Java<eot>2问题:《红楼梦》的作者', 'java', 'java']

In [30]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['“谁知盘中餐,粒粒皆辛苦', '“汗滴禾下土”', '“谁知盘中餐,粒粒皆辛苦']

In [31]:
ask_gpt('烤鸭的主要原料是什么？')

['鸭肉', '鸭肉', '鸭油']

In [32]:
ask_gpt('把大象放冰箱总共分几步？')

['8步', '2步', '5步']

In [33]:
ask_gpt('珠穆朗玛峰有多高？')

['世界最高峰', '8844.43米', '8848米']

In [34]:
ask_gpt('世界上最高的山是什么？')

['喜马拉雅山', '喜马拉雅山', '喜马拉雅山']

In [35]:
ask_gpt('中国有多少人？')

['2亿多人', '5亿5千万人', '12亿']

In [36]:
ask_gpt('日本有多少人？')

['日本', '800万', '100多万']

In [37]:
ask_gpt('苹果手机好还是华为手机好？')

['华为手机好', '华为', '华为']

# 推理

In [38]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小红。
问题:小红和小明去吃饭
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明<eot>请问一下我是开通
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小红。
小明:为什么不去呢?
--------------------


# 英文问答例子

In [39]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟pi<eot>怎么样
这个可以的,
--------------------
默写英文:
狗dog
猫cat
鸟dog
猴key
鸡d
--------------------
默写英文:
狗dog
猫cat
鸟bird
猪labour

--------------------


# 默写古诗例子

In [40]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望明
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望明
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望明
--------------------


# 不同的角色对话生成例子

In [41]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“大嘴,你啥时候把掌柜的叫来?”
李大嘴:“
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“各回各家,各找各妈!”
佟湘玉:“哥哥,
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“各找各妈!”
李大嘴:“各找各妈!”
--------------------


In [42]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“各位同僚!”
蒋百里:“各位同仁!”

--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“你走你的阳关道,我走我的独木桥!”
李大
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“老李,你说这人叫什么名字?”
李嘉图:
--------------------


In [43]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“咦,你怎么又不走了?”
李大嘴:“有你
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“我们家小贝和莫子轩就像是两只被丢弃的小狗,而我们家
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“各睡各的,各忙各的,各做各的,各睡各的,各
--------------------


In [44]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“天行健,君子以自强不息!”
李四光:“人
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我是个不成熟的人。”
李大嘴:“你比我成熟
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我说小爷,别以为你长得好看就可以去泡妞,还真以为你是
--------------------


# 问答的例子

In [45]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
中国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
中国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
德国
--------------------


In [46]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是宋
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明代
--------------------


In [47]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------


In [48]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是多少
珠穆朗玛峰的高度(
--------------------
珠穆朗玛峰的高度(米)是7,9,6,3,2,1
--------------------
珠穆朗玛峰的高度(米)是5,848.00<eot>珠穆朗玛峰
--------------------


In [49]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是多少
好像是2米13<eot>姚明的
--------------------
姚明的身高(米)是多少
2.16米<eot>姚明的身高(
--------------------
姚明的身高(米)是多少
姚明的身高(米)是多少
--------------------


# 算数例子

In [50]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=7
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=7
--------------------


In [51]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------


# ???的例子

In [52]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
赤炎这魔焰
青虹这玄冰
黑光这混沌玄火
青龙这玄冥火
巨象
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
玄冰这九天玄冰
玄冰这九天玄冰
玄天这九幽冥冰
玄天这九幽冥冰
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
九阳这太乙真水
冰霜这太乙真火
烈焰这太乙真火
青炎这太乙
--------------------


# 写作文例子

In [53]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是什么呢
爱情就是男女之间的那种相互的感觉<eot>爱情是什么呢?爱情就是互相依靠,互相理解,互相信任,互相尊重<eot>爱情是什么呢?爱情就是互相信任,互相尊重,互相理解,互相信任
--------------------
爱情是一场谁也输不起的战争的作者是⁇⁇<eot>爱情是一场谁也输不起的战争的连载网站是红袖添香网<eot>爱情是一场谁也输不起的战争的连载网站是红袖添香网<eot>爱情是
--------------------
爱情是什么?很多人都说爱情就是幸福。可是,现实生活中,有多少人能看懂这幸福?又有多少人将自己的爱情当做幸福?对于爱情,我们可以将它当做一种享受,可以将
--------------------


In [54]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。黛玉因是第一次来,怕见生人,只在外面隔着帘子,探首张望。只听里面是一个女人说话声音,便知是贾母娘娘之妻,宝钗姊妹之母。黛玉不禁微微皱了眉头,因又听宝钗在说道:“我记得咱们这里只有那边有几间房子,也是咱们奶奶的,怎么这会子都搬到这里来了?”<eot>随着企业间的合作,企业之间的竞争也日益加剧,而作为服务贸易行业,面对日益激烈的市场竞争,中国企业如何在激烈的市场竞争中,立于不败之地?中国市场巨大,吸引着越来越多的企业纷纷加入到服务贸易的行业中来。现在,越来越多的企业开始认识到,要想在竞争中立于不败之地,企业之间的合作就显得十分重要。一个成功的企业,都是有一定的优势
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。上得楼来,早有丫鬟引着,一路进了大厅,有几案桌凳,却无丫鬟小厮,只有一个老嬷嬷在堂中侍候。黛玉见了这个老嬷嬷,便知端王,凤姐,惜春,史湘云等几个家中人皆在此处,也不在别处,就随了老嬷嬷,又到了上房,只听“嗤嗤”一阵响,焙茗便进来说:“贾奶奶请了。”黛玉听了,忙起来,便往上房行去。上得二层,只见上房的正中,摆着一个紫檀木的大柜,上面镶着无数的金叶,宝云也坐在那边,一见黛玉来了,忙起身让坐。黛玉笑道:“我正等着你呢。”便下了楼来,到了荣府中。
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。到了地方,黛玉便问“你们在这里站着干什么?”众人道:“太太吩咐,有什么话,只管进去说话。”黛玉道:“我就是进去说了一句话,你们就站着,有什么要紧。”众嬷嬷笑道:“我们又不是外人,还怕什么?”黛玉笑道:“你们只管站着,我只管往里走。”众嬷嬷

# 对话例子

In [55]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“为什么”
A:“我想吃火锅”
B:“你不是在考试吗”
A:“我在考试”
B:“哦”
A:“想吃火锅”
B
--------------------
A:“今天我想吃火锅”
B:“那我来煮给你吃吧”
A:“这样才像我啊”
我:“我的重点是你有没有诚意”
A:“我又不是你的佣人”
我:“
--------------------
A:“今天我想吃火锅”
B:“哦,我想吃火锅”
A:“你为什么不想吃火锅”
B:“因为我想吃麻辣烫”
A:“你为什么不想吃火锅”
B:“因为我想吃火锅
--------------------


In [56]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“你确定?”
A:“当然了”
B:“那我们现在去哪儿?”
A:“回家”
A:“那就走吧”
B:“嗯”

--------------------
A:“跟我一起去看电影吧”
B:“好啊”
A:“你在哪啊?”
B:“我就在你旁边啊”
A:“卧槽,你在给我看电影啊!”
B:“我在陪你
--------------------
A:“跟我一起去看电影吧”
B:“嗯,我也想看。”
A:“你太好了,我看了你最近的新电影,很精彩哦。”
B:“你看的什么电影?”
A:“你看了
--------------------


# 对联例子

In [57]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [58]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫写诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联马东作
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作画
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联孔子做人
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联王安石
--------------------


In [59]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联呵护友情

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联守护平安
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联珍惜友情

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联珍惜感情

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联守住家庭

--------------------


In [60]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联月月升平
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联事事如意
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联心情愉悦
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天挨揍

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天好心情

--------------------


In [61]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联四海皆兄弟

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联人间处处家

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联四海归我意
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联人生在世

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联万物皆我种
--------------------


# 名人名言

In [62]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“我所提倡的是我所厌恶的,然而我对于我所厌恶的却总是不得不厌恶的。”这句话对于我们来说似乎有些沉重,但其实我们也都不愿意和身边的人或事物产生矛盾,可是随着
--------------------
鲁迅曾经说过:“一个人最宝贵的,不是他的生命,而是他的头脑;一个人最重要的,不是他的见解,而是他的主张;一个人最容易忘记的,不是他的痛苦,而是他的
--------------------
鲁迅曾经说过:“什么是‘民族的脊梁’?不是‘民族的’,而是‘民族的’中国人。”
对于中国文化来说,“脊梁”与“脊梁”的含义、作用是不同的。
“脊梁”
--------------------


In [63]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“我不会给你任何东西,但我有一个小小的条件,我会把这两个字母写在你的名字的左边。”当我们开始在意生活的质量的时候,其实,我们已经开始在意
--------------------
爱因斯坦曾经说过:“你的想象力越丰富,你的创造力越强。”想象力丰富的人,就像一块未经雕琢的璞玉,蕴含着无尽的魅力。
在人生的旅途中,常常有很多人因为没有良好
--------------------
爱因斯坦曾经说过:“我的世界观是从我小时候所形成的,我的兴趣爱好也是从小就养成的。在我的成长过程中,我的世界观,我的兴趣爱好都得到了发展。”正是这种兴趣爱好和
--------------------


In [64]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“人在大自然面前是平等的。”如果人类为了生存而放弃了自己的本性,那么自然界就会报复人类。现在,大自然正在报复人类。今天,我们的社会正在报复我们。从前,在
--------------------
牛顿曾经说过:“时间就是一个著名的、奇妙的东西。它是一个大魔术师,谁能解开这个谜团,谁就能成为一位伟大的学者。”在他看来,时间是一把“双刃剑”,一方面
--------------------
牛顿曾经说过:“用什么工具可以证明一个东西的存在?我现在告诉你,就用你的眼睛吧。”他就用眼睛来看这个问题。他发现,这个问题很简单,就是要把你的眼睛变成一个放大镜,可以
--------------------


In [65]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“我们的世界太疯狂了,因为我们不懂得珍惜生命。”这话是对的,但我认为这句话也不全对。珍惜生命就是珍惜我们现在所拥有的一切,而不是等到我们的
--------------------
佛祖曾经说过:“我从没说过我是谁,我说的是我心中的佛,我心中的佛就是佛祖”。一个人,如果能放下心中的执着,真正去信仰佛教,那么,他的生命将会得到最深层
--------------------
佛祖曾经说过:“我的一切,都是为了世界的和平。”我们要是不好好保护地球,我们自己的星球也不行。我们要是不好好保护地球,我们自己的星球也不行。
要是有一天,
--------------------


In [66]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“如果你拥有梦想,就等于拥有了一切。”这的确是至理名言。在乔布斯的梦想中,我们可以看到他对未来的愿景,也可以看到他的奋斗历程。我们可以看到他身为管理
--------------------
乔布斯曾经说过:“我们是一家以创新为中心的公司,创新一直都是我们的工作。”在他的领导下,苹果从最开始的一家硬件制造商成长为一家可以影响市场格局的科技公司,其原因之一就在
--------------------
乔布斯曾经说过:“在你的所有的Macbook电脑上都要用到它。”
Macbook用久了,就习惯了。<eot>看了这么久的书,还是没活明白<eot>我喜欢吃西红柿炒鸡蛋<eot>电脑
--------------------


# 菜谱

In [71]:
ret = sample(
    tokenizer, gpt, '''青椒肉丝的做法：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

青椒肉丝的做法:1、葱姜切丝,青椒去籽,切丝,木耳泡发,切丝;2、锅中放油,放葱姜丝爆香,再放青椒,翻炒出香味,放入肉丝翻炒,
--------------------
青椒肉丝的做法:1、把肉丝放在开水中⁇一下捞出控净水。<eot>青椒肉丝的做法:1、把肉丝放在开水中⁇一下捞出控净水。2、锅中放少许油,爆香
--------------------
青椒肉丝的做法:1、猪肉丝里加入料酒、生抽、盐、鸡精、葱姜、蛋清搅拌均匀;2、锅里倒油,下肉丝滑散,捞出控油;3、青椒切成
--------------------


In [72]:
ret = sample(
    tokenizer, gpt, '''圣女果炒茶叶蛋的做法：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

圣女果炒茶叶蛋的做法:1、鸡蛋磕入碗中,加入精盐,用筷子打散;2、锅洗净后,放入油,油热后倒入鸡蛋液,迅速划散,至鸡蛋成蛋花;3
--------------------
圣女果炒茶叶蛋的做法:是鸡蛋打散,盐、味精、料酒等调料入锅<eot>圣女果炒茶叶蛋的做法:是鸡蛋打散,盐、味精、料酒等调料入锅<eot>圣女果炒茶叶蛋的做法
--------------------
圣女果炒茶叶蛋的做法:是放入适量的食盐<eot>圣女果炒茶叶蛋的做法:是放入适量的食盐<eot>圣女果炒茶叶蛋的做法:是放入适量的食盐<eot>圣女果炒茶叶蛋的做法:是放入适量
--------------------


In [73]:
ret = sample(
    tokenizer, gpt, '''中国四大发明是''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国四大发明是什么
中国四大发明是造纸术、火药、指南针和印刷术<eot>中国四大发明是什么?<eot>【四种中国传统神秘发明】
--------------------
中国四大发明是指哪些
中国四大发明是指我国古代的四大发明,即指南针、造纸术、火药和印刷术。造纸术是我国古代最伟大的发明之一,是世界四大发明之一。指南针的发明,使我国
--------------------
中国四大发明是指哪四大发明
中国四大发明指的是火药、指南针、造纸术、活字印刷术。<eot>为什么苹果手机不能玩王者荣耀?王者荣耀和LOL有什么
--------------------


In [74]:
ret = sample(
    tokenizer, gpt, '''上联：瑞风播福泽，事业昌盛千家乐
下联：''',
    3, 50, top_k=50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

上联:瑞风播福泽,事业昌盛千家乐
下联:祥云接福来,事业兴旺千佰姓
横批:福禄寿
上联:瑞风播福泽,事业昌盛千家乐
下联:祥云接福来,事业兴旺千佰姓

--------------------
上联:瑞风播福泽,事业昌盛千家乐
下联:春燕入宅,春风扑面来万户欢
横批:步步高升<eot>我这个人很低调,喜欢一个人独处。所以想在春节假期,给父母多陪陪。在这个假期,我准备去上海
--------------------
上联:瑞风播福泽,事业昌盛千家乐
下联:江波锁蛟龙,子子孙孙万代兴
横批:万事如意
此联为“瑞风传福”四字,可横批为“万事如意”。
1、“瑞风传
--------------------
